<a href="https://colab.research.google.com/github/avdhoot001/23B0060_AIC/blob/main/T_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are a lot of improvements i can do here but this is what i could do due to time constraints. Another reason was i exhausted all my credits for the open ai api key from all my family members account. I dont know how. This should not be the reason for such a poor project of mine but it is what it is. Really sorry

In [ ]:
#install required dependancies
!pip install sentence-transformers
!pip install faiss-cpu
!pip install torch
!pip install datasets
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import faiss
from transformers import T5ForConditionalGeneration, T5Tokenizer


In [ ]:
# Load a pre-trained sentence transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Example dataset
passages = [
    "The Eiffel Tower is located in Paris.",
    "The Great Wall of China is visible from space.",
    "The capital of Japan is Tokyo.",
    "The largest ocean on Earth is the Pacific Ocean.",
    "Mount Everest is the highest mountain in the world.",
    "my name is avdhoot",
    "Avdhoot is the strongest man on earth the world has ever seen",
    "God is Great",
]

# Compute embeddings for the dataset
passage_embeddings = embedder.encode(passages, convert_to_tensor=True)

# Create FAISS index
index = faiss.IndexFlatL2(passage_embeddings.shape[1])
index.add(passage_embeddings.cpu().detach().numpy())


In [ ]:
# Load a pre-trained T5 model and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def retrieve_passages(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_tensor=True)
    query_embedding = query_embedding.cpu().detach().numpy()

    # Search the FAISS index
    D, I = index.search(query_embedding, top_k)

    # Retrieve the top-k passages
    results = [passages[idx] for idx in I[0]]
    return results


In [ ]:
def chatbot_response(query):
    retrieved_passages = retrieve_passages(query)
    combined_input = " ".join(retrieved_passages)

    # Prepare the input for the T5 model
    t5_input_text = f"question: {query} context: {combined_input}"
    input_ids = t5_tokenizer.encode(t5_input_text, return_tensors='pt')

    # Generate the response using T5 model
    outputs = t5_model.generate(input_ids)
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [ ]:
query = "what is capital of japan?"
response = chatbot_response(query)
print(f"User: {query}")
print(f"Chatbot: {response}")


User: what is capital of japan?
Chatbot: Tokyo


There are a lot of improvements i can do here but this is what i could do due to time constraints. Anothet reason was i exhausted all my credits for the open ai api key from all my family members account. I dont know how. This should not be the reason for such a poor project of mine but it is what it is